# Introduction to Monte Carlo Simulations

## Retirement Fund

Suppose you have a retirement fund currently valued at $100,000 invested in the S&P500 ETF. 

You plan to retire in 30 years.

How much should you expect to have in your retirement account


## Some global variables

In [ ]:
present_value = 100000
expected_return = .095
time_horizon = 30


# Deterministic Approach

e.g. a fixed rate of return of 9.5% for 30 years

In [ ]:
def compound(principal, rate, time, n = 1):
    return principal * (1 + rate/n) ** (time*n)

ending_balance = compound(present_value, expected_return, time_horizon)

print("{:,.0f}".format(ending_balance))

To produce a year on year table showing the nature of compound interest

In [ ]:
ending_balance = 0
curr_balance = present_value

print("{:10s} {:15s}".format("Year", "Ending Balance"))
print("-" * 26)
for year in range(1, time_horizon + 1):
    ending_balance = curr_balance * (1 + expected_return)
    print("{:<10d} {:15,.0f}".format(year, ending_balance))
    curr_balance = ending_balance


## Non Deterministic Approach

But can we reliably earn 9.5% every year?

We need to incorporate volatility, e.g. 18.5


In [ ]:
import numpy.random as npr

volatility = .185

ending_balance = 0
curr_balance = present_value

print("{:10s}  {:15s}".format("Year", "Ending Balance"))
print("-" * 26)

for year in range(1,time_horizon + 1):
    year_return = npr.normal(expected_return, volatility)
    ending_balance = curr_balance * (1 + year_return)
    print("{:<10d} {:>15,.0f}".format(year, ending_balance))
    curr_balance = ending_balance

## Which one to choose

No right answer. 

What can be done is to run the above simulation many times and to get an idea of the types of expected returns.

e.g. run for 50,000 iterations

This allows us to make probability statements about how such an investment will perform

# Monte Carlo Simulation

For the same scenario above
> <BR>
> $100,000 initial investment <BR>
> 9.5% expected return <BR>
> 18.5% volatility <BR>
> 30 year time frame (horizon) <BR>
> <BR>

But run **50,000** simulations

In [ ]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)

sns.set()
%matplotlib inline

In [ ]:
iterations = 50000
starting = present_value
ending = 0


Create a 2 dimensional array (iterations, horizon) to store the results

i.e. 50,000 iterations with 1 column for each year

In [ ]:
returns = np.zeros((iterations, time_horizon))

Run 50,000 times

In [ ]:
for t in range(iterations):
    for year in range(time_horizon):
        returns[t][year] = npr.normal(expected_return, volatility)

In [ ]:
# look at a random simulation
returns[42]        

## Compute Values

Same structure as above but calcualting the value rather than the return

In [ ]:
portfolio = np.zeros((iterations, time_horizon))
for iteration in range(iterations):
    latest = starting
    for year in range(time_horizon):
        ending = latest * (1 + returns[iteration,year])
        portfolio[iteration,year] = ending
        latest = ending

In [ ]:
# Pick one at random
portfolio[42]

## Reshape the data

Change the portfolio into a DataFrame

Transpose to easier understand the data

In [ ]:
portfolio = pd.DataFrame(portfolio).transpose()

display(portfolio[list(range(5))].head())

display(portfolio[list(range(5))].tail())

In [ ]:
display(pd.concat(objs=[portfolio[list(range(5))].head(3), portfolio[list(range(5))].tail(3)]))

## Look at final year only

Notice the mean is pretty close to the deterministic approach.

But notice the max and min, std and the various percentiles



In [ ]:
portfolio.iloc[29].describe()

## Visualize

The vertical red line is the portfolio at the 50th percentile - considerably lower in value that the mean portfolio. 

The overall distribution is lognormal.

In [ ]:
final_year = portfolio.iloc[29]

# Filter out portfolios with final value > $10M
lt_10M = final_year < 10 * 1000 * 1000

fig, ax = plt.subplots(figsize=(12,8))

chart = sns.histplot(data = final_year[lt_10M], bins=100, kde=True)
chart.set_xticks([x*1000*1000 for x in range(0,10)])
chart.set_xticklabels([f"{x}" for x in range(0,10)])

plt.axvline(final_year[lt_10M].median(), color='r')

plt.title('Monte Carlo Simulation - $100,000 invested for 30yrs -- expected return 9.5%, volatility 18.5%')
plt.xlabel('Final Value ($M)')
plt.ylabel('Count of Simulations');

plt.savefig('Monte Carlo 1.png')

## Make some statements

For example
the minimum values at the 

In [ ]:
percentiles = [1,5,10]
v99,v95,v90 = np.percentile(final_year, percentiles)

print(f'There is a 99% chance that the investment will be at least {v99:,.0f} at maturity')
print(f'There is a 95% chance that the investment will be at least {v95:,.0f} at maturity')
print(f'There is a 90% chance that the investment will be at least {v90:,.0f} at maturity')